<a href="https://colab.research.google.com/github/teator/FinEng/blob/main/Capstone/WQU_capstone_Using_VIX_Based_RSI_Measures_to_Time_Markets_(Quantopian_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---   ---   ---   ---   ---   ---   ---
## WorldQuant University :-: MScFE (C18-S4)
## WQU capstone: Using VIX Based RSI Measures to Time Markets

Tea Toradze

June-August 2020
---   ---   ---   ---   ---   ---   ---

In [ ]:
# Libraries
import numpy as np
from quantopian.pipeline import Pipeline, CustomFilter
from quantopian.algorithm import attach_pipeline, pipeline_output
from quantopian.pipeline.data.quandl import cboe_vix
from quantopian.pipeline.factors import RSI, SimpleMovingAverage, Latest
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.data import morningstar



'''
The VIX RSI trading system as per Connors, L. and Alvarez, C.(2008) also described in:
http://systemtradersuccess.com/vix-rsi/

The Rules
1. Price must be above its 200-day moving average
2. RSI(2) on price must be below 35
3. Buy when RSI(2) of the VIX is above 65
4. Exit when RSI(2) of the price rises above 65


Opposite Rules for short (SELL):
1. Price must be below its 200-day moving average
2. RSI(2) on price must be above 65
3. Sell when RSI(2) of the VIX is below 35
4. Exit when the RSI(2) of the price drops below 35
'''


## Initialise SPY securities
def initialize(context):
    context.spy = symbol('SPY')

    context.invested = False
    context.leverage = 1.95
    context.short = False

    schedule_function(trade, date_rules.every_day(), time_rules.market_open())

    attach_pipeline(my_pipeline(context), 'my_pipeline')


## The main pipeline to build RSI, Moving Average and VIX RSI data
def my_pipeline(context):

    spy_filter = SidInList(sid_list = (
            context.spy.sid,
            )
        )

    close = Latest(
            inputs=[USEquityPricing.close],
            mask = spy_filter,
        )

    ma_200 = SimpleMovingAverage(
            inputs = [USEquityPricing.close],
            window_length = 200,
            mask = spy_filter,
        )

    rsi_2 = RSI(
            inputs = [USEquityPricing.close],
            window_length = 2,
            mask = spy_filter,
        )

    vix_rsi_2 = RSI(
            inputs = [cboe_vix.vix_close],
            window_length = 2,
            mask = spy_filter,
        )

    pipe = Pipeline()

    pipe.add(close, 'close')
    pipe.add(ma_200, 'ma_200')
    pipe.add(rsi_2, 'rsi_2')
    pipe.add(vix_rsi_2, 'vix_rsi_2')

    pipe.set_screen(spy_filter)

    return pipe


## Simulate trade function
def trade (context, data):

    results = pipeline_output('my_pipeline')

    spy = context.spy
    spy_close = results.loc[spy].close
    spy_ma200 = results.loc[spy].ma_200
    spy_rsi = results.loc[spy].rsi_2

    vix_rsi = results.iloc[0].vix_rsi_2


    if not context.invested:
        if vix_rsi > 65 and spy_rsi < 35 and spy_close > spy_ma200:
            order_target_percent(spy, context.leverage)
            context.invested = True
            context.short = False
        elif vix_rsi < 35 and spy_rsi > 65 and spy_close < spy_ma200:
            order_target_percent(spy, -context.leverage)
            context.invested = True
            context.short = True
    else:
        if (context.short and spy_rsi < 35 and vix_rsi > 10) or (not context.short and spy_rsi > 65 and vix_rsi < 10):
            order_target(spy, 0)
            context.invested = False


    # Record algo results:
    if context.invested:
        if context.short:
            position = -100.0
        else:
            position = 100.0
    else:
        position = 0.0

    record(vix_rsi = vix_rsi,
           spy_rsi = spy_rsi,
           spy_ma200 = spy_ma200,
           spy = spy_close,
           pos = position,
          )


## Custom filter (
class SidInList(CustomFilter):
    inputs = []
    window_length = 1
    params = ('sid_list',)

    def compute(self, today, assets, out, sid_list):
        out[:] = np.in1d(assets, sid_list)



In [ ]:
## Tear sheet
bt = get_backtest('5f20799f2fd87446ba870f7e')
bt.create_full_tear_sheet()

